# Baseline

In [5]:
# Sonar 信号分类 基准模型

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 随机数重复性 设定
seed = 7
np.random.seed(seed)

# 加载数据
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
df = pd.read_csv(url, header=None)
dataset = df.values

# 划分特征与标签
X = dataset[:, 0: 60].astype(float)
Y = dataset[:, 60]

# 类别编码成数字形式 0 or 1
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# 基准 60-60-30-1
def create_baseline():
    # 创建模型
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(30, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # 模型编译
    # 学习算法 stochasitic gradient  descent
    # lr: float >=0. learning rate
    # momentum:  float >= 0. Parameter updates momentum.
    # decay: float >= 0. Learning rate decay over each update.
    # nesterov: boolean. Whether to apply Nesterov momentum.
    sgd = SGD(lr=0.01, momentum=0.6, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

np.random.seed(seed)

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, nb_epoch=500,
                                         batch_size=18, verbose=0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Baseline: {0:.2f}% ({1:.2f}%)".format(results.mean()*100, results.std()*100))

D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:42: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, activation="relu", kernel_initializer="normal", input_dim=60)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal")`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Baseline: 54.29% (3.33%)


# sgd 参考

- [Unsupervised Feature Learning and Deep Learning Tutorial](http://ufldl.stanford.edu/tutorial/supervised/OptimizationStochasticGradientDescent/)
- [CS231n Convolutional Neural Networks for Visual Recognition](http://cs231n.github.io/neural-networks-3/)

# Using Dropout on the Visible Layer

In [6]:
# dropout in the input layer with weight constraint
def create_model():
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60, )))
    model.add(Dense(60, init='normal', activation='relu', W_constraint=maxnorm(3)))
    model.add(Dense(30, init='normal', activation='relu', W_constraint=maxnorm(3)))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # compile
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

np.random.seed(seed)

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=500, 
                                         batch_size=18, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Visible: {0:.2f}% ({1:.2f}%)".format(results.mean()*100, results.std()*100))

D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, activation="relu", kernel_initializer="normal", kernel_constraint=<keras.con...)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal", kernel_constraint=<keras.con...)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Visible: 86.52% (4.26%)


# Using dropout on the hidden layters

In [7]:
# dropout in hidden layers with weight constraint

def create_model():
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu',
                   W_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(30, init='normal', activation='relu', W_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # compile
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=500, 
                                         batch_size=18, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Visible: {0:.2f}% ({1:.2f}%)".format(results.mean()*100, results.std()*100))

D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, activation="relu", kernel_initializer="normal", kernel_constraint=<keras.con..., input_dim=60)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal", kernel_constraint=<keras.con...)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Visible: 82.68% (3.83%)


# dropout 

源 论文:

[srivastava14a.pdf](http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)